In [72]:
#%matplotlib widget
import backtrader as bt
import backtrader.analyzers as btanalyzers
import rba_tools as rba
import pandas as pd
import numpy as np
#import matplotlib
from datetime import datetime
from datetime import date
from datetime import timedelta

In [2]:
class MaCrossStrategy(bt.Strategy):

    def __init__(self):
        ma_fast = bt.ind.SMA(period = 10)
        ma_slow = bt.ind.SMA(period = 50)

        self.crossover = bt.ind.CrossOver(ma_fast, ma_slow)

    def next(self):
        if not self.position:
            if self.crossover > 0:
                self.buy()
        elif self.crossover < 0:
            self.close()

In [3]:
cerebro = bt.Cerebro()

In [4]:
binance = rba.getBinanceExchange()
dataframe = rba.get_DataFrame(['ETH/BTC'], binance, '1/1/18', '12/31/19')
data = bt.feeds.PandasData(dataname=dataframe,
                               # datetime='Date',
                               nocase=True,
                               )
cerebro.adddata(data)

In [5]:
dataframe2 = rba.get_DataFrame(['LTC/BTC'], binance, '1/1/18', '12/31/19')
data2 = bt.feeds.PandasData(dataname=dataframe2,
                               # datetime='Date',
                               nocase=True,
                               )
cerebro.adddata(data2)

In [6]:
cerebro.addstrategy(MaCrossStrategy)

0

In [7]:
cerebro.broker.setcash(1000000.0)

In [8]:
cerebro.addsizer(bt.sizers.PercentSizer, percents = 10)

In [9]:
#cerebro.addanalyzer(btanalyzers.SharpeRatio, _name = 'sharpe')
#cerebro.addanalyzer(btanalyzers.Transactions, _name = 'trans')
#cerebro.addanalyzer(btanalyzers.Returns, _name = 'returns')
cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name = 'trades') #good for summary stats on trades
cerebro.addanalyzer(btanalyzers.PositionsValue, _name = 'pos', cash=True) #this is a good one. Shows position values over time
cerebro.addanalyzer(btanalyzers.AnnualReturn, _name = 'annReturns')

In [10]:
back = cerebro.run()

In [11]:
cerebro.broker.getvalue()

971104.7165789388

In [12]:
for x in back[0].analyzers:
    print(x)

In [13]:
mytrades = back[0].analyzers.trades.get_analysis()

In [14]:
def pintdictdict(indict,indent=0):
    indent += 1
    for x in indict:
        print('\n','   ' * indent,x,end='',sep='')
        val = indict[x]
        if isinstance(val,bt.utils.autodict.AutoOrderedDict):
            pintdictdict(val,indent)
        else:
            print('=',val,sep='',end='')

In [15]:
pintdictdict(mytrades)


   total
      total=8
      open=0
      closed=8
   streak
      won
         current=0
         longest=1
      lost
         current=7
         longest=7
   pnl
      gross
         total=-28895.28342106143
         average=-3611.910427632679
      net
         total=-28895.28342106143
         average=-3611.910427632679
   won
      total=1
      pnl
         total=17762.26659598281
         average=17762.26659598281
         max=17762.26659598281
   lost
      total=7
      pnl
         total=-46657.55001704424
         average=-6665.364288149177
         max=-10683.950281394302
   long
      total=8
      pnl
         total=-28895.28342106143
         average=-3611.910427632679
         won
            total=17762.26659598281
            average=17762.26659598281
            max=17762.26659598281
         lost
            total=-46657.55001704424
            average=-6665.364288149177
            max=-10683.950281394302
      won=1
      lost=7
   short
      total=0
      pnl


In [16]:
annReturns = back[0].analyzers.annReturns.get_analysis()
annReturns

OrderedDict([(2018, 0.01613699725686879), (2019, -0.04431713518895353)])

In [17]:
pos = back[0].analyzers.pos.get_analysis()
for d in [date(2019, 2, 17) + timedelta(days=x) for x in range(15)]:
    print(d,'=',pos[d])

2019-02-17 = [0.0, 0.0, 998638.932260116]
2019-02-18 = [0.0, 0.0, 998638.932260116]
2019-02-19 = [98102.75252712643, 0.0, 898705.5554860452]
2019-02-20 = [99968.11854810042, 0.0, 898705.5554860452]
2019-02-21 = [98880.43377655827, 0.0, 898705.5554860452]
2019-02-22 = [99366.8186129727, 0.0, 898705.5554860452]
2019-02-23 = [102386.68050939191, 0.0, 898705.5554860452]
2019-02-24 = [94342.62359946109, 0.0, 898705.5554860452]
2019-02-25 = [96242.73139446469, 0.0, 898705.5554860452]
2019-02-26 = [95582.63768790226, 0.0, 898705.5554860452]
2019-02-27 = [94513.66002545298, 0.0, 898705.5554860452]
2019-02-28 = [94831.68088003164, 0.0, 898705.5554860452]
2019-03-01 = [94572.4537968877, 0.0, 898705.5554860452]
2019-03-02 = [92728.46732916268, 0.0, 898705.5554860452]
2019-03-03 = [91646.12744593278, 0.0, 898705.5554860452]


In [18]:
for x in back[0].stats:
    print(x)

In [19]:
datatrades = back[0].stats[1]
broker = back[0].stats[0]

In [20]:
for x in back[0].stats:
    print(type(x))
    print('x.plotinfo.plot =',x.plotinfo.plot)
    print('x.plotinfo.plotskip =',x.plotinfo.plotskip)
    print('x.plotinfo.subplot =',x.plotinfo.subplot)

<class 'backtrader.observers.broker.Broker'>
x.plotinfo.plot = True
x.plotinfo.plotskip = False
x.plotinfo.subplot = True
<class 'backtrader.observers.trades.DataTrades'>
x.plotinfo.plot = True
x.plotinfo.plotskip = False
x.plotinfo.subplot = True


In [21]:
myobs = back[0].stats[1]
myobs

In [22]:
isinstance(myobs ,bt.observers.trades.DataTrades)

True

In [23]:
ind = myobs

In [24]:
for lineidx in range(ind.size()):
            print('lineidx =',lineidx)
            line = ind.lines[lineidx]
            linealias = ind.lines._getlinealias(lineidx)
            print('linealias =',linealias)
            lineplotinfo = getattr(ind.plotlines, '_%d' % lineidx, None)
            print('ind.plotlines =',ind.plotlines)
            print('lineplotinfo =',lineplotinfo)
            if not lineplotinfo:
                lineplotinfo = getattr(ind.plotlines, linealias, None)
                print('2 lineplotinfo =',lineplotinfo)
            if not lineplotinfo:
                lineplotinfo = AutoInfoClass()
                print('3 lineplotinfo =',lineplotinfo)
            pltmethod = lineplotinfo._get('_method', 'plot')
            print('pltmethod =',pltmethod)
            if pltmethod != 'plot':
                toskip += 1 - lineplotinfo._get('_plotskip', False)

lineidx = 0
linealias = 
ind.plotlines = <backtrader.metabase.AutoInfoClass_pl_LineSeries_pl_LineIterator_pl_DataAccessor_pl_ObserverBase_pl_Observer_pl_DataTrades_44e4388a6017422b9937c44e07029ee2 object at 0x0000020C7B98CCA0>
lineplotinfo = None
2 lineplotinfo = <backtrader.metabase.AutoInfoClass_44e4388a6017422b9937c44e07029ee2_ object at 0x0000020C7B98CCD0>
pltmethod = plot
lineidx = 1
linealias = 
ind.plotlines = <backtrader.metabase.AutoInfoClass_pl_LineSeries_pl_LineIterator_pl_DataAccessor_pl_ObserverBase_pl_Observer_pl_DataTrades_44e4388a6017422b9937c44e07029ee2 object at 0x0000020C7B98CCA0>
lineplotinfo = None
2 lineplotinfo = <backtrader.metabase.AutoInfoClass_44e4388a6017422b9937c44e07029ee2_ object at 0x0000020C7B98CCD0>
pltmethod = plot


In [25]:
for line in myobs.lines:
    npary = np.frombuffer(line.array)
    print(npary[~np.isnan(npary)])
    print(len(npary[~np.isnan(npary)]))

[ 17762.26659598  -4050.46051866  -5153.06062077  -9919.81319643
  -8832.42793599 -10683.95028139  -2157.22029591  -5860.61716789]
8
[]
0


In [26]:
npary = np.frombuffer(myobs.lines[0].array)
npary

array([nan, nan, nan, ..., nan, nan, nan])

In [27]:
npary[~np.isnan(npary)]

array([ 17762.26659598,  -4050.46051866,  -5153.06062077,  -9919.81319643,
        -8832.42793599, -10683.95028139,  -2157.22029591,  -5860.61716789])

In [28]:
myline = myobs.lines[0]
myline

In [29]:
myline.array[0:1000]

array('d', [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 17762.26659598281, nan, nan, nan, nan, nan, nan, -4050.460518664533, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,

In [30]:
data = {'trades': myline.array}
df = pd.DataFrame(data=data)
df

,trades
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
1458,NaN
1459,NaN
1460,NaN
1461,NaN


In [31]:
df.loc[np.isnan(df['trades']) == False]

,trades
156,17762.266596
163,-4050.460519
321,-5153.060621
393,-9919.813196
448,-8832.427936
537,-10683.950281
668,-2157.220296
697,-5860.617168


In [32]:
def get_trades_from_cerebro_run(cerebro_run):
    data = None
    for strat in cerebro_run[0].stats:
        if isinstance(strat, bt.observers.trades.DataTrades):
            for line in strat.lines:
                npary = np.frombuffer(line.array)
                if len(npary[~np.isnan(npary)]) > 0:
                    data = npary
    return data
result = get_trades_from_cerebro_run(back)
print(result)
print(result[~np.isnan(result)])

[nan nan nan ... nan nan nan]
[ 17762.26659598  -4050.46051866  -5153.06062077  -9919.81319643
  -8832.42793599 -10683.95028139  -2157.22029591  -5860.61716789]


In [44]:
def get_broker_cash_from_cerebro_run(cerebro_run):
    data = None
    for strat in cerebro_run[0].stats:
        if isinstance(strat, bt.observers.broker.Broker):
            #assumes we want line 0 which I think is safe for Broker?
            npary = np.frombuffer(strat.lines[0].array)
            data = npary
    return data
result = get_broker_cash_from_cerebro_run(back)
print(result)
print(len(result))

[1000000. 1000000. 1000000. ...      nan      nan      nan]
1463


[ 0.  0.  0. ... nan nan nan]


In [47]:
def get_pos_analysis(cerebro_run):
    for analyzer in cerebro_run[0].analyzers:
        if isinstance(analyzer, btanalyzers.PositionsValue):
            return analyzer.get_analysis().copy()

In [69]:
def get_cash_including_position(cerebro_run):
    #this assumes that PositionsValue analyzer exists with parameter cash=True (this is not the default) and headers=False (this is the default)
    pos_analysis = get_pos_analysis(cerebro_run)
    data = []
    for key, value in pos_analysis.items():
        data.append(sum(value))
    return np.round_(np.array(data),2)
result = get_cash_including_position(back)
result

array([1000000.  , 1000000.  , 1000000.  , 1000000.  , 1000000.  ,
       1000000.  , 1000000.  , 1000000.  , 1000000.  , 1000000.  ,
       1000000.  , 1000000.  , 1000000.  , 1000000.  , 1000000.  ,
       1000000.  , 1000000.  , 1000000.  , 1000000.  , 1000000.  ,
       1000000.  , 1000000.  , 1000000.  , 1000000.  , 1000000.  ,
       1000000.  , 1000000.  , 1000000.  , 1000000.  , 1000000.  ,
       1000000.  , 1000000.  , 1000000.  , 1000000.  , 1000000.  ,
       1000000.  , 1000000.  , 1000000.  , 1000000.  , 1000000.  ,
       1000000.  , 1000000.  , 1000000.  , 1000000.  , 1000000.  ,
       1000000.  , 1000000.  , 1000000.  , 1000000.  , 1000000.  ,
       1000000.  , 1000000.  , 1000000.  , 1000000.  , 1000000.  ,
       1000000.  , 1000000.  , 1000000.  , 1000000.  , 1000000.  ,
       1000000.  , 1000000.  , 1000000.  , 1000000.  , 1000000.  ,
       1000000.  , 1000000.  , 1000000.  , 1000000.  , 1000000.  ,
       1000000.  , 1000000.  , 1000000.  , 1000000.  , 1000000

In [71]:
def get_percent_cash_change(cerebro_run):
    cash_vals = get_cash_including_position(cerebro_run)
    start_val = cash_vals[0]
    return np.round((cash_vals / start_val - 1) * 100,2)
result = get_percent_cash_change(back)
print(result)

[ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.41  0.62  0.74  0.26  0.52  0.63  0.79  0.81  0.69
  0.92  0.98  1.79  1.92  2.24  2.12  1.85  2.03  1.9   1.82  1.87  1.9
  2.38  2.38  2.27  2.48  2.23  2.39  2.48  2.37  2.25  1.85  1.36  1.7
  1.57  1.79  1.45  0.63  1.19  1.14  1.38  1.35  1.4   1.83  1.63  1.78
  1.78  1.78  1.78  1.83  1.63  1.6   1.37  1.37  1.37  1.37  1.37  1.3

In [64]:
result

array([1000000.        , 1000000.        , 1000000.        ,
       1000000.        , 1000000.        , 1000000.        ,
       1000000.        , 1000000.        , 1000000.        ,
       1000000.        , 1000000.        , 1000000.        ,
       1000000.        , 1000000.        , 1000000.        ,
       1000000.        , 1000000.        , 1000000.        ,
       1000000.        , 1000000.        , 1000000.        ,
       1000000.        , 1000000.        , 1000000.        ,
       1000000.        , 1000000.        , 1000000.        ,
       1000000.        , 1000000.        , 1000000.        ,
       1000000.        , 1000000.        , 1000000.        ,
       1000000.        , 1000000.        , 1000000.        ,
       1000000.        , 1000000.        , 1000000.        ,
       1000000.        , 1000000.        , 1000000.        ,
       1000000.        , 1000000.        , 1000000.        ,
       1000000.        , 1000000.        , 1000000.        ,
       1000000.        ,